In [1]:
from p2pfl.node import Node
import time

In [2]:
n=2
nodes = []
for i in range(n):
    node = Node(host="localhost")
    node.start()
    nodes.append(node)
    


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpm36g3sfg
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmpm36g3sfg/_remote_module_non_sriptable.py
INFO:root:Nodo a la escucha en localhost 49448
INFO:root:Nodo a la escucha en localhost 49449


In [3]:
for i in range(len(nodes)-1):
    nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
    time.sleep(0.1)


INFO:root:Conexión aceptada con ('localhost', 49449)


In [4]:
# Check if they are connected
for node in nodes:
    print(len(node.neightboors) == n-1)
    

True
True


In [5]:
nodes[0].set_start_learning()

INFO:root:Broadcasting start learning...
INFO:root:Broadcasting model to all clients...
DEBUG:root:Processing message: ['START_LEARNING', '1']
INFO:root:Broadcasting model to all clients...
Params received
INFO:root:Model added (1/1)
INFO:root:Agregating models.
Params received
INFO:root:Model added (1/1)
INFO:root:Agregating models.
DEBUG:root:Agregating done.
DEBUG:root:Agregating done.
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']


In [6]:
for node in nodes:
    print(node.learner.get_parameters())

OrderedDict([('l1.weight', tensor([[ 0.0076,  0.0168,  0.0119,  ...,  0.0137,  0.0116, -0.0264],
        [ 0.0091, -0.0061,  0.0198,  ..., -0.0304,  0.0156, -0.0088],
        [-0.0169,  0.0016,  0.0072,  ..., -0.0175,  0.0239,  0.0038],
        ...,
        [-0.0095,  0.0037, -0.0278,  ..., -0.0056,  0.0133,  0.0032],
        [-0.0254, -0.0005, -0.0087,  ..., -0.0085, -0.0029, -0.0136],
        [ 0.0046, -0.0002,  0.0040,  ...,  0.0204, -0.0056, -0.0150]])), ('l1.bias', tensor([ 4.7275e-03,  1.1386e-02, -7.7219e-03, -6.6124e-03, -1.3960e-02,
         6.4580e-03, -2.0499e-02,  3.0505e-02, -6.7417e-03, -2.0314e-02,
         1.6286e-02, -2.8283e-03,  6.8733e-03,  1.1851e-02,  2.3236e-03,
         2.4841e-02, -2.8154e-02, -6.4387e-03, -1.0641e-02, -2.3200e-03,
        -8.9899e-03, -9.1787e-03,  1.0154e-02, -1.4106e-02, -4.9394e-03,
         2.1370e-02, -4.3896e-03,  2.6545e-02,  2.3548e-02, -1.7543e-02,
        -9.7825e-03, -1.6980e-02,  2.1966e-03,  1.1090e-02,  1.2185e-02,
         1.309

DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']


DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']


In [7]:
for node in nodes:
    print(node.round)

1
1
1
1
1


In [11]:
nodes[0].set_stop_learning()

INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning
INFO:root:Stopping learning


In [9]:

from p2pfl.node import Node
from p2pfl.learning.model import MyNodeLearning
from p2pfl.agregator import FedAvg    
from collections import OrderedDict
import torch
import time

def test_round_result_equal(): # Parametrizar esto

    # Node Creation
    n=2
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning()
    
    # Waiting
    time.sleep(0.1)

    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:

                a = model[layer]
                b = node.learner.get_parameters()[layer]
                print(a)
                print(b)
                assert torch.eq(a, b).all()
                
test_round_result_equal()

INFO:root:Nodo a la escucha en localhost 49580
INFO:root:Nodo a la escucha en localhost 49581
INFO:root:Conexión aceptada con ('localhost', 49581)
INFO:root:Broadcasting start learning...
INFO:root:Broadcasting model to all clients...
DEBUG:root:Processing message: ['START_LEARNING', '1']
INFO:root:Broadcasting model to all clients...
tensor([[-0.0057, -0.0065,  0.0113,  ...,  0.0104,  0.0321,  0.0135],
        [ 0.0229, -0.0138,  0.0141,  ...,  0.0170,  0.0205,  0.0045],
        [-0.0128, -0.0190, -0.0264,  ..., -0.0206, -0.0170, -0.0047],
        ...,
        [ 0.0021, -0.0212,  0.0319,  ...,  0.0195, -0.0281, -0.0085],
        [ 0.0239,  0.0084,  0.0049,  ..., -0.0049,  0.0273, -0.0169],
        [ 0.0165,  0.0120, -0.0048,  ...,  0.0215,  0.0310, -0.0216]])
Params received
tensor([[-0.0162, -0.0139,  0.0090,  ...,  0.0062,  0.0304, -0.0054],
        [ 0.0202,  0.0013, -0.0011,  ..., -0.0178, -0.0216, -0.0060],
        [-0.0347,  0.0182, -0.0119,  ..., -0.0147,  0.0321,  0.0041],
   

AssertionError: 

DEBUG:root:Agregating done.
Params received
INFO:root:Model added (1/1)
INFO:root:Agregating models.
DEBUG:root:Agregating done.
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: ['BEAT']
DEBUG:root:Processing message: 